# Introduction

- Douwe Maljers (15264424)
- Justin van Egmond (15011178)
- Noor Verbrugge (15168298)

Mentale gezondheid onder studenten is de afgelopen jaren een groeiend maatschappelijk thema geworden. De combinatie van allemaal verschillende omstandigheden, zoals prestatiedruk, sociale verwachtingen en onzekerheid over de toekomst maakt deze groep bijzonder kwetsbaar. In deze data-analyse richten we ons op het identificeren van factoren die samenhangen met mentale gezondheid, met als doel het onthullen van mogelijke risico’s en beschermende factoren binnen het studentenleven.

Op basis van enquêtegegevens van studenten wereldwijd, verkennen we de verbanden tussen mentale gezondheid en verschillende leefomstandigheden, gedragingen en gevoelens. Daarbij kijken we naar zowel interne als externe factoren die het welzijn kunnen beïnvloeden. De persectieven waaruit wij naar de mentale gezondheid gaan kijken zijn, social media gebruik, slaap, sport gewoonten ....

Wereldwijd is de populatie met mentale stoornissen significant gestegen. In de grafiek boven is per land de trend te zien. Vandaar dat dit onderzoek nog meer van belang is. Vrouwen en mannen zijn beide slachtoffer van deze chronische ziekte. 

In dit onderzoek worden meerdere standpunten onderbouwd aan de hand van data visualisaties, specifiek gericht op de invloeden op metale gezondheid onder studenten. Dit wordt gedaan met behulp van verschillende datasets van diverse groepen studenten. Tevens wordt een dataset van het Institute for Health Metrics and Evaluation gebruikt om de grootschaligheid van het wereldwijde probleem van mentale stoornissen en verminderde mentale gesteldheid in kaart te brengen.

In [1]:
import pandas as pd
import plotly.graph_objects as go
import pycountry

# === Gegevens inladen ===
pad_csv = "datasets/IHME_mental_real.csv"
gegevens = pd.read_csv(pad_csv)

# === Landen omzetten naar ISO3-codes ===
def naar_iso3(naam):
    try:
        return pycountry.countries.lookup(naam).alpha_3
    except LookupError:
        return None

gegevens["landcode"] = gegevens["location"].apply(naar_iso3)

# === Waarden als percentages weergeven ===
gegevens["percentage"] = gegevens["val"]
gegevens.loc[gegevens["metric"] == "Percent", "percentage"] *= 100
gegevens.loc[gegevens["metric"] == "Rate", "percentage"] *= 100

# === Parameters ===
geslachten = ["Both", "Female", "Male"]
jaren = sorted(gegevens["year"].unique())
min_waarde = 0
max_waarde = gegevens["percentage"].max()

# === Kaartlagen aanmaken voor alle combinaties van geslacht en jaar ===
kaartlagen = []
zichtbaarheid = {}
for geslacht in geslachten:
    for jaar in jaren:
        sleutel = f"{geslacht}_{jaar}"
        selectie = gegevens[(gegevens["sex"] == geslacht) & (gegevens["year"] == jaar)]
        trace = go.Choropleth(
            locations=selectie["landcode"],
            z=selectie["percentage"],
            text=selectie["location"],
            colorscale="Reds",
            zmin=min_waarde,
            zmax=max_waarde,
            colorbar_title="Prevalentie (%)",
            visible=False,
            name=sleutel,
        )
        kaartlagen.append(trace)
        zichtbaarheid[sleutel] = len(kaartlagen) - 1

# === Startinstellingen ===
start_geslacht = "Both"
start_jaar = jaren[0]
start_sleutel = f"{start_geslacht}_{start_jaar}"
kaartlagen[zichtbaarheid[start_sleutel]].visible = True

# === Dropdownmenu voor geslacht (labels in het Nederlands) ===
keuzemenu_knoppen = []
for geslacht in geslachten:
    label_nl = {
        "Both": "Beide",
        "Female": "Vrouw",
        "Male": "Man"
    }[geslacht]
    
    knop = dict(
        label=label_nl,
        method="update",
        args=[
            {"visible": [False] * len(kaartlagen)},
            {"title": f"Mentale stoornissen – {label_nl}"},
        ],
    )
    for jaar in jaren:
        sleutel = f"{geslacht}_{jaar}"
        index = zichtbaarheid.get(sleutel)
        if jaar == start_jaar and index is not None:
            knop["args"][0]["visible"][index] = True
    keuzemenu_knoppen.append(knop)

# === Sliderstappen per jaar ===
slider_stappen = []
for jaar in jaren:
    stap = dict(
        method="update",
        args=[
            {"visible": [False] * len(kaartlagen)},
            {"title": f"Mentale stoornissen – Beide"},  # Defaulttitel blijft consistent
        ],
        label=str(jaar),
    )
    sleutel = f"{start_geslacht}_{jaar}"
    index = zichtbaarheid.get(sleutel)
    if index is not None:
        stap["args"][0]["visible"][index] = True
    slider_stappen.append(stap)

# === Figuur opbouwen ===
figuur = go.Figure(
    data=kaartlagen,
    layout=go.Layout(
        title_text="Mentale stoornissen – Beide",
        geo=dict(showframe=False, showcoastlines=True, projection_type="natural earth"),
        updatemenus=[
            dict(
                active=0,
                buttons=keuzemenu_knoppen,
                x=0.05,
                y=1.12,
                xanchor="left",
                yanchor="top",
            )
        ],
        sliders=[
            dict(
                active=0,
                pad={"t": 50},
                x=0.1,
                len=0.8,
                xanchor="left",
                yanchor="top",
                currentvalue={"prefix": "Jaar: ", "font": {"size": 16}},
                steps=slider_stappen,
            )
        ]
    )
)

figuur.show()


ModuleNotFoundError: No module named 'pycountry'

> *Figure 1: Per geslacht het percentage mentale stoornissen wereldwijd*